In [1]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter
import os
import sys
from ipywidgets import FloatProgress
from IPython.display import clear_output
from IPython.display import display

from tqdm import tqdm
from tqdm import trange

In [2]:
LR = 1e-3
gym.envs.register(
    id='MyCartPole-v0',
    entry_point='gym.envs.classic_control:CartPoleEnv',
    tags={'wrapper_config.TimeLimit.max_episode_steps' : 2000},
)
env = gym.make("MyCartPole-v0")
env.reset()
goal_steps = 2000
score_requirement = 100
initial_games = 1000000

[2017-07-27 13:53:45,209] Making new env: MyCartPole-v0


In [3]:
def some_random_games():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
            env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if(done):
                break

In [4]:
#some_random_games()

In [5]:
def initial_population():
    train_data = []
    scores = []
    accepted_scores = []
    progressbar = FloatProgress(min=0, max=initial_games)
    display(progressbar)
    for index, _ in tqdm(enumerate(range(initial_games)), total=initial_games):
        progressbar.value = index
        #percent = int((index / initial_games) * 100)
        #if not index % 10000:
            #print("\r" + "Progress:" + "[" + "\u2588"*((percent // 10) + 1) + (" "*(9 - (percent // 10))) + "]" + str(percent) + "% - " + str(index))
        
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            #
            action = random.randrange(0,2)

            #action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if len(prev_observation) > 0:
                game_memory.append([prev_observation,action])
                
            prev_observation = observation
            score += reward
            if done:
                break
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                train_data.append([data[0], output])
        env.reset()
        scores.append(score)
    train_data_save = np.array(train_data)
    np.save('saved.npy', train_data_save)
    print('Average accepted score: ', mean(accepted_scores))
    print('Median accepted score: ', median(accepted_scores))
    print(Counter(accepted_scores))
    
    return train_data
    

In [6]:
#initial_population()

In [7]:
def neural_network_model(input_size):
    network = input_data(shape=[None, input_size, 1], name='input')
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 1024, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    
    model = tflearn.DNN(network, tensorboard_dir='log')
    
    return model

In [8]:
def train_model(train_data, model=False):
    x = np.array([i[0] for i in train_data]).reshape(-1,len(train_data[0][0]), 1)
    y = [i[1] for i in train_data]
    
    if not model:
        model = neural_network_model(input_size= len(x[0]))
        
    model.fit({'input':x}, {'targets':y}, n_epoch=3, snapshot_step=500, show_metric=True,run_id='openaistuff')
    
    return model

In [9]:
def play_game(visible=True):
    scores = []
    choices = []

    for each_game in range(10):
        score = 0
        game_memory = []
        prev_obs = []
        env.reset()
        for _ in range(goal_steps):
            if visible:
                env.render()
            if len(prev_obs) == 0:
                action = random.randrange(0,2)
            else:
                action = np.argmax(model.predict(prev_obs.reshape(-1, len(prev_obs), 1))[0])
            choices.append(action)

            new_obs, reward, done, info = env.step(action)
            prev_obs = new_obs
            game_memory.append([new_obs, action])
            score += reward
            if done:
                break
        scores.append(score)
    average_score = sum(scores)/len(scores)
    if visible:
        print('Scores: ',scores)
        print('Average Score ', average_score)
        print('Choice 0: {}, Choice 1: {}'.format(choices.count(0)/len(choices), choices.count(1)/len(choices)))
    return average_score


In [ ]:
#training_data = initial_population()
#model = train_model(training_data)
#average_score = play_game(False)
#while average_score < 300:
#    training_data = initial_population()
#    model = train_model(training_data, model)
#    average_score = play_game(False)
#play_game()

In [ ]:
training_data = initial_population()

Widget Javascript not detected.  It may not be installed or enabled properly.


  1%|          | 11433/1000000 [00:20<34:35, 476.30it/s]

In [ ]:
model = train_model(training_data)

In [ ]:
modelscore = play_game()

In [ ]:
modelscore

In [ ]:
modelscore = int(round(modelscore))
if(modelscore > 200):
    path = "models/" + str(modelscore) + "/"
    if not os.path.exists(path):
        os.makedirs(path)
    model.save(path + str(modelscore) + ".model")